# 4.2.1 Densely Connected NN

In the last notebook, we jumped into training on a CNN. However, we quickly realized that history information seemed redundant and present information may be the most valuable. Given that, we decided to try a feed forward network and the results were interesting enough to warrant furtherr investigation. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy')
y_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/y_train.npy')

In [3]:
X_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_test.npy')
y_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/y_test.npy')

In [4]:
np.unique(y_train, return_counts=True)

(array([0., 1., 2.]), array([34245, 16007,  9748]))

In [5]:
# Undersample to balance classes for training set
h = np.where(y_train == 0)[0]
b = np.where(y_train == 1)[0]
s = np.where(y_train == 2)[0]

hi = np.random.choice(h, size=9000, replace=False)
bi = np.random.choice(b, size=9000, replace=False)
si = np.random.choice(s, size=9000, replace=False)

In [6]:
ind = np.concatenate([hi,bi,si])
ind.shape

(27000,)

In [7]:
np.unique(y_train[ind], return_counts=True)

(array([0., 1., 2.]), array([9000, 9000, 9000]))

In [8]:
X_train[ind].shape

(27000, 1, 116, 60)

In [9]:
y_train = y_train[ind]
X_train = X_train[ind]

In [10]:
X_train.shape, y_train.shape

((27000, 1, 116, 60), (27000, 1))

In [11]:
X_train.shape = (27000, 116, 60)
X_train.shape

(27000, 116, 60)

In [12]:
X_train = X_train[:, :, :10]
X_train.shape

(27000, 116, 10)

In [13]:
#X_train.shape = (27000, 116)

In [14]:
# decrease test size for runtime and memory concerns
h = np.where(y_test == 0)[0]
b = np.where(y_test == 1)[0]
s = np.where(y_test == 2)[0]

hi = np.random.choice(h, size=4000, replace=False)
bi = np.random.choice(b, size=4000, replace=False)
si = np.random.choice(s, size=4000, replace=False)

In [15]:
indt = np.concatenate([hi,bi,si])
indt.shape

y_test = y_test[indt]
X_test = X_test[indt]
y_test.shape, X_test.shape

((12000, 1), (12000, 1, 116, 60))

In [16]:
X_test = X_test.reshape(12000, 116, 60)

In [17]:
X_test = X_test[:, :, :10]

In [18]:
#X_test.shape = (12000, 116)

In [19]:
np.unique(y_test, return_counts=True)[1]/y_test.shape[0]

array([0.33333333, 0.33333333, 0.33333333])

Options:  
    1) decrease training set size and run more epochs  
    2) Change image size to make them shorter  
    3) Change filters, kernal, and layers   

#### Attempt 3 columns only

So far, the super simple feed forward network seems to perform just as well as the cnn. 

In [23]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(116, 10)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 27000 samples, validate on 12000 samples
Epoch 1/50
27000/27000 [==============================] - 4s 166us/sample - loss: 1.0940 - accuracy: 0.3601 - val_loss: 1.0972 - val_accuracy: 0.3555
Epoch 2/50
27000/27000 [==============================] - 4s 152us/sample - loss: 1.0761 - accuracy: 0.3905 - val_loss: 1.1081 - val_accuracy: 0.3618
Epoch 3/50
27000/27000 [==============================] - 4s 154us/sample - loss: 1.0730 - accuracy: 0.3997 - val_loss: 1.1049 - val_accuracy: 0.3567
Epoch 4/50
27000/27000 [==============================] - 4s 160us/sample - loss: 1.0690 - accuracy: 0.4062 - val_loss: 1.1021 - val_accuracy: 0.3580
Epoch 5/50
27000/27000 [==============================] - 4s 160us/sample - loss: 1.0701 - accuracy: 0.4075 - val_loss: 1.0994 - val_accuracy: 0.3515
Epoch 6/50
27000/27000 [==============================] - 4s 154us/sample - loss: 1.0823 - accuracy: 0.4028 - val_loss: 1.1035 - val_accuracy: 0.3658
Epoch 7/50
27000/27000 [==========================

In [20]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(X_train)

In [27]:
predictions[4]

array([0.07750326, 0.92002517, 0.00247163], dtype=float32)

In [22]:
np.argmax(predictions)

13971

In [60]:
predictions = probability_model.predict(X_test)

In [61]:
np.argmax(predictions[0])

0

In [62]:
predictions[0]

array([9.8680556e-01, 1.2953308e-02, 2.4112873e-04], dtype=float32)

### implement data augmentation and dropout 